# Import

In [61]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [4]:
import load_script

# Loading the dataset

In [5]:
train_dataset,target=load_script.load_dataset(train=True)

In [7]:
train_dataset[0].shape

torch.Size([28, 50])

# Defining the network

In [73]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,5)
        
        self.full1=nn.Linear(10 * 12 * 23,100)
        self.full2=nn.Linear(100,2)
        
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x=x.view(-1)
        x=F.relu(self.full1(x))
        x=self.full2(x)
        
        return x

In [74]:
net=Net()
net

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (full1): Linear(in_features=2760, out_features=100, bias=True)
  (full2): Linear(in_features=100, out_features=2, bias=True)
)

# Training

In [95]:
net=Net()

optimizer = optim.SGD(net.parameters(), lr=0.00000001)

for epoch in range(25):
    
    total_loss=0
    for train_element,target_element in zip(train_dataset,target):
        
        train_element=train_element.view(1,1,28,-1)
        
        optimizer.zero_grad()
        
        out=net(train_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    print('Epoch:',epoch,'Loss:',total_loss)

Epoch: 0 Loss: 5843.057990685105
Epoch: 1 Loss: 2270.1250189081766
Epoch: 2 Loss: 1538.555721821729
Epoch: 3 Loss: 1364.921992721036
Epoch: 4 Loss: 1299.0027284063399
Epoch: 5 Loss: 1255.4545667357743
Epoch: 6 Loss: 1218.121690088883
Epoch: 7 Loss: 1184.330945700407
Epoch: 8 Loss: 1153.3849327513017
Epoch: 9 Loss: 1125.01882298477
Epoch: 10 Loss: 1098.9332205085084
Epoch: 11 Loss: 1074.8635383564979
Epoch: 12 Loss: 1052.7136357016861
Epoch: 13 Loss: 1032.3133752569556
Epoch: 14 Loss: 1013.5041393321007
Epoch: 15 Loss: 995.9329852517694
Epoch: 16 Loss: 979.583095267415
Epoch: 17 Loss: 964.4177967514843
Epoch: 18 Loss: 950.2931800307706
Epoch: 19 Loss: 937.1102567203343
Epoch: 20 Loss: 924.8635688498616
Epoch: 21 Loss: 913.5005934138317
Epoch: 22 Loss: 902.8629421496298
Epoch: 23 Loss: 892.8735466797953
Epoch: 24 Loss: 883.5208691989537
